In [1]:
import numpy as np
from sortedcontainers import SortedDict

In [2]:
moves=np.array([[1,3],[0,2,4],[1,5],[1,4,6],[1,3,5,7],[2,4,8],[3,7],[6,4,8],[5,7]])
initial = [5,4,0,6,1,8,7,3,2] # 0 marks where it is empty
final = [1,2,3,8,0,4,7,6,5]
visited = SortedDict()

In [3]:
def Move(state, pos):
    empty = state.index(0)
    state[empty], state[pos] = state[pos], state[empty]
    return state

In [4]:
def PrintBoard(state):
    print (np.asarray(state).reshape((3,3)))

In [5]:
def Numerify(state):
    return int("".join(map(str, state[:]))) 

In [6]:
def BuscaCega(source, queue):
    counter = 0
    visited[Numerify(source)]={'moves': 0, 'parent': None}
    queue.insert(0,source)   
    while (len(queue) > 0):
        u = queue.pop()
        m = visited[Numerify(u)]['moves']
        empty = u.index(0) # where is empty
        for v in moves[empty]:
            state = Move(u[:], v) # u[:] makes a copy of u
            if not(visited.__contains__(Numerify(state))):
                visited[Numerify(state)]={'moves':m+1, 'parent': u}
                if (Numerify(state) == Numerify(final)):
                        print (f'{counter} evaluations.')
                        print (visited[Numerify(state)])
                        return
                else:
                    counter = counter + 1
                    queue.insert(0,state)

In [7]:
BuscaCega(initial, [])

279355 evaluations.
{'moves': 25, 'parent': [1, 2, 3, 8, 4, 0, 7, 6, 5]}


In [8]:
def Trace(state):
    stack = []
    stack.append(state)
    parent = visited[Numerify(state)]['parent']
    while (parent is not None):
        stack.insert(0, parent)
        parent = visited[Numerify(parent)]['parent']
    for i, s in enumerate(stack):
        print(f'move #{i})
        PrintBoard(s)
    

In [9]:
Trace(final)

[[5 4 0]
 [6 1 8]
 [7 3 2]]
[[5 0 4]
 [6 1 8]
 [7 3 2]]
[[0 5 4]
 [6 1 8]
 [7 3 2]]
[[6 5 4]
 [0 1 8]
 [7 3 2]]
[[6 5 4]
 [1 0 8]
 [7 3 2]]
[[6 0 4]
 [1 5 8]
 [7 3 2]]
[[0 6 4]
 [1 5 8]
 [7 3 2]]
[[1 6 4]
 [0 5 8]
 [7 3 2]]
[[1 0 4]
 [6 5 8]
 [7 3 2]]
[[1 4 0]
 [6 5 8]
 [7 3 2]]
[[1 4 8]
 [6 5 0]
 [7 3 2]]
[[1 4 8]
 [6 5 2]
 [7 3 0]]
[[1 4 8]
 [6 5 2]
 [7 0 3]]
[[1 4 8]
 [6 0 2]
 [7 5 3]]
[[1 4 8]
 [0 6 2]
 [7 5 3]]
[[1 0 8]
 [4 6 2]
 [7 5 3]]
[[1 8 0]
 [4 6 2]
 [7 5 3]]
[[1 8 2]
 [4 6 0]
 [7 5 3]]
[[1 8 2]
 [4 6 3]
 [7 5 0]]
[[1 8 2]
 [4 6 3]
 [7 0 5]]
[[1 8 2]
 [4 0 3]
 [7 6 5]]
[[1 8 2]
 [0 4 3]
 [7 6 5]]
[[1 0 2]
 [8 4 3]
 [7 6 5]]
[[1 2 0]
 [8 4 3]
 [7 6 5]]
[[1 2 3]
 [8 4 0]
 [7 6 5]]
[[1 2 3]
 [8 0 4]
 [7 6 5]]
